In [1]:
import random
import torch
import sys
from ImportantConfig import Config
from sql2fea import TreeBuilder, value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
from PGUtils import pgrunner
import numpy as np
import pandas as pd
from sql_feature.workload_embedder import PredicateEmbedderDoc2Vec

sys.path.append('/home/ubuntu/project/HyperQO')

config = Config()
# sys.stdout = open(config.log_file, "w")
random.seed(0)

with open(config.queries_file) as f:
    import json

    queries = json.load(f)

tree_builder = TreeBuilder()
sql2vec = Sql2Vec()
# 这里的 input_size 必须为偶数！
value_network = SPINN(head_num=config.head_num, input_size=36, hidden_size=config.hidden_size, table_num=50,
                          sql_size=config.sql_size, attention_dim=30).to(config.device)
for name, param in value_network.named_parameters():
    from torch.nn import init

    if len(param.shape) == 2:
        init.xavier_normal(param)
    else:
        init.uniform(param)

treenet_model = TreeNet(tree_builder, value_network)

mask = (torch.rand(1, config.head_num, device=config.device) < 0.9).long()

train = pd.read_csv('./information/train.csv', index_col=0)
queries = train['query'].values

workload_embedder = PredicateEmbedderDoc2Vec(queries, 20, pgrunner)

train.head()

/tmp/ipykernel_27503/1544516627.py:40: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(param)
/tmp/ipykernel_27503/1544516627.py:38: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(param)
2024-03-01 12:44:33,948 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-03-01 12:44:34,198 : INFO : adding document #10000 to Dictionary<134 unique tokens: ['Index_Scan_customer_demographics_Filter_cd_gender=ANDcd_marital_status=ANDcd_education_status=Index_Cond_cd_demo_sk=store_sales.ss_cdemo_sk', 'Seq_Scan_date_dim_Filter_d_year=', 'Seq_Scan_promotion_Filter_p_channel_email=ORp_channel_event=', 'Index_Scan_customer_address_Filter_ca_country=ANDca_state=ANYORca_state=ANYORca_state=ANYANDca_state=ANYANDstore_sales.ss_net_profit>=ANDstore_sales.ss_net_profit<=ORca_state=ANYANDstore_sales.ss_net_profit>=ANDstore_sales.ss_net_profit<=ORca_state=ANYANDstore_sales.ss_net_profit>=AND

,query,cost_no_index,cost_dta,cost_reduction,cost_reduction_ratio
0,"select i_item_id, \n avg(ss_quantity) ...",108534.81,69925.31,38609.50,0.355734
1,"select sum (ss_quantity)\n from store_sales, s...",147137.68,86652.87,60484.81,0.411076
2,"select i_item_id\n ,i_item_desc \n ...",50155.33,3944.83,46210.50,0.921348
3,with ss_items as\n (select i_item_id item_id\n...,162566.89,839.12,161727.77,0.994838
4,"select asceding.rnk, i1.i_product_name best_p...",274060.77,48.66,274012.11,0.999822


In [2]:
x = torch.tensor(train.index)
y = torch.tensor(train['cost_reduction_ratio'].values)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 例如，均方误差损失
optimizer = treenet_model.optimizer  # 例如，Adam 优化器

Batch_Size = 32
torch_dataset = Data.TensorDataset(x, y)

In [9]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
train_set, val_set = train_test_split(torch_dataset, test_size=0.2, shuffle = True)
len(train_set),len(val_set)

(11520, 2880)

In [14]:
loader_val = Data.DataLoader(dataset=val_set,
                                 batch_size=Batch_Size//4,
                                 shuffle=True)
loader_val=[x for x in loader_val]

In [17]:
batch_x_val,batch_y_val=loader_val[batch_num]
batch_x_val,batch_y_val

(tensor([ 7876,  9119,  7251,  2479,   935,  1411, 10724, 12925]),
 tensor([0.0769, 0.6167, 0.1156, 0.0727, 0.9403, 0.3241, 0.0397, 0.2933],
        dtype=torch.float64))

In [21]:
run_cnt=1
list_pred=[]
list_loss=[]
list_pred_val=[]
list_loss_val=[]
list_batch_loss=[]
list_batch_loss_val=[]
# 训练循环
batch_num=0
for epoch in range(1):  # 例如，训练多个 epochs
    loader = Data.DataLoader(dataset=train_set,
                                 batch_size=Batch_Size,
                                 shuffle=True)
    loader_val = Data.DataLoader(dataset=val_set,
                                 batch_size=Batch_Size//4,
                                 shuffle=True)
    loader_val=[x for x in loader_val]
    for batch_x, batch_y in loader:
        optimizer.zero_grad()  # 每个批次前先清零梯度
        batch_loss = 0
        batch_loss_val=0
        # training process
        for num in range(Batch_Size):
            sql = queries[batch_x[num]]
            target_value = batch_y[num]
            plan_json = pgrunner.getCostPlanJson(sql)
            sql_vec = workload_embedder.get_embedding([sql])

            # 计算损失
            loss, pred_val = treenet_model.train(plan_json, sql_vec, target_value, mask,is_train=True)
            list_loss.append(loss)
            list_pred.append(pred_val)
            print("training count {} : train loss : {}, pred_val : {}, target_value : {},  diff : {}".format(run_cnt,loss, pred_val,target_value,abs(pred_val-target_value)))
            batch_loss += loss  # 累积批次损失
            run_cnt+=1
        list_batch_loss.append(batch_loss)
        print("batch loss : {}".format(batch_loss / Batch_Size))
        # val process  4:1的比例进行验证
        for num in range(Batch_Size//4):
            # valid process
            batch_x_val,batch_y_val=loader_val[batch_num]
            sql = queries[batch_x_val[num]]
            target_value = batch_y_val[num]
            plan_json = pgrunner.getCostPlanJson(sql)
            sql_vec = workload_embedder.get_embedding([sql])

            # 计算损失
            loss, pred_val = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=False)
            list_loss_val.append(loss)
            list_pred_val.append(pred_val)
            print("valid epo : {}, train loss : {}, pred_val : {}, target_value : {},  diff : {}".format(batch_num,loss, pred_val,target_value,abs(pred_val-target_value)))
            batch_loss_val +=loss
        list_batch_loss_val.append(batch_loss)
        print("valid batch loss : {}".format(batch_loss_val / (Batch_Size//4)))
        batch_num+=1  #记录批次batch

KeyboardInterrupt: 

In [5]:
torch.save(treenet_model.value_network.state_dict(), 'model.pth')

In [6]:
# 加载模型参数
model =TreeNet(tree_builder, value_network)
model.value_network.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [8]:
model.train(plan_json, sql_vec, target_value, mask, is_train=False)

(tensor(0.0175, dtype=torch.float64, grad_fn=<SmoothL1LossBackward0>),
 tensor(0.4807, grad_fn=<SqueezeBackward0>))

In [ ]:
import matplotlib.pyplot as plt


list_loss=[float(x) for x in list_loss]
list_pred=[(x) for x in list_pred]
# x轴数据
x = list(range(len(list_pred)))
 
# y轴第一条曲线数据
y1 = list_loss[::50]
 
# y轴第二条曲线数据
y2 = list_pred[::50]
 
# 创建子图对象
fig, ax = plt.subplots()
 
# 绘制第一条曲线
ax.plot(x, y1, label='Curve 1')
 
# 绘制第二条曲线
ax.plot(x, y2, label='Curve 2')
 
# 添加标题、x轴和y轴标签
plt.title('Two Curves Plot')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
 
# 显示图例
plt.legend()
 
# 显示图形
plt.show()

In [10]:
res=pd.DataFrame()
res['loss']=[float(x) for x in list_loss]
res['pred']=[float(x) for x in list_pred]
res.to_csv('training_result.csv')
res.head()

,loss,pred
0,0.010964,0.502861
1,0.110074,0.397846
2,0.002929,0.451062
3,0.022763,0.461958
4,0.002416,0.467955
